<a href="https://colab.research.google.com/github/obinnachike/Emotion_Detection_DL/blob/main/EMOTION_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d muhammadhananasghar/human-emotions-datasethes

In [ ]:
!unzip "/content/human-emotions-datasethes.zip" -d "/content/dataset/"

In [ ]:
train_directory = "copy_the_train_path"
val_directory = "copy_the_test_path"
CLASS_NAMES = ['angry', 'happy', 'sad']
CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 256,
    "LEARNING_RATE": 0.001,
    "N_EPOCHS": 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASS": 3,

}

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels = "inferred",
    label_mode = "categorical"
    class_names = CLASS_NAMES,
    color_mode = "rgb",
    batch_size = 32,
    image_size = (256, 256)
    shuffle = True,
    seed = 99,
    #validation_split = None,
   # subset = 'trianing',
    #interpolation = "bilinear",
    #follow_links = False,
    #crop_to_aspect_ratio = False
)

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels = "inferred",
    label_mode = "int"
    class_names = CLASS_NAMES,
    color_mode = "rgb",
    batch_size = 32, or batch_size =  CONFIGURATION["BATCH_SIZE"]
    image_size = (256, 256) or image_size = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"])
    shuffle = True,
    seed = 99,
    #validation_split = None,
   # subset = 'trianing',
    #interpolation = "bilinear",
    #follow_links = False,
    #crop_to_aspect_ratio = False
)

In [ ]:
for i in val_dataset.take(1):
  print(i)

In [ ]:
plt.figure(figsize =(12,12))

for images, labels in train_dataset.take(1):
  for i in range(16):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(images[i]/225.)
    plt.title(CLASS_NAMES[tf.argmax(labels[i], axis = 0).numpy()])
    plt.axis('off')

In [ ]:
training_dataset = (
    train_dataset.prefetch(tf.AUTOTUNE)
)

In [ ]:
VAL_dataset = (
    VAL_dataset.prefetch(tf.AUTOTUNE)
)

In [ ]:
resize_rescale_layers = tf.keras.Sequential([
    Resizing(IM_SIZE, IM_SIZE),
    Rescaling(1./225)
])

# remember to import rescale and resize from keras.layers

In [ ]:
lenet_model = tf.keras.Sequential([
                             InputLayer(input_shape = (None, None, 3)), resize_rescale_layers,




                             Conv2D(filters = CONFIGURATION["N_FILTERS"], kernel_size = CONFIGURATION["KERNEL_SIZE"], strides = CONFIGURATION[N_STRIDES], padding = 'valid', activation = 'relu',kernel_regularizer = L2(CONFIGURATION[])),
                             BatchNormalization(),
                             MaxPooling2D(pool_size = CONFIGURATION[POOL_SIZE], strides = CONFIGURATION[N_STRIDES]*2), Dropout(rate = CONFIGURATION["HP_DROPOUT"]),

                             Conv2D(filters = CONFIGURATION["N_FILTERS"]*2 + 4, kernel_size = CONFIGURATION["KERNEL_SIZE"], strides = CONFIGURATION[N_STRIDES], padding = 'valid', activation = 'relu',kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
                             BatchNormalization(),
                             MaxPooling2D(pool_size = CONFIGURATION[POOL_SIZE], strides = CONFIGURATION[N_STRIDES]*2), Dropout(rate = CONFIGURATION["HP_DROPOUT"]),

                             Flatten(),
                             Dense(CONFIGURATION["N-DENSE_1"], activation = 'relu', kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
                             BatchNormalization(), Dropout(rate = CONFIGURATION["HP_DROPOUT"]),

                             Dense(CONFIGURATION["N-DENSE_2"], activation = 'relu', kernel_regularizer = L2(CONFIGURATION["REGULARIZATION_RATE"])),
                             BatchNormalization(),

                             Dense(CONFIGURATION["NUM_CLASSES"], activation = 'softmax')
])

lenet_model.summary()

In [ ]:
# LOSS FUNCTION
loss_function = CategoricalCrossEntropy()
#loss_function = CategoricalCrossEntropy(from_logits = True)

In [ ]:
metrics = [CategoricalAccuracy(name = "accuracy"),  TopKCategoricalAccuracy(k = 2, name = "top_k_accuracy")]

In [ ]:
lenet_model.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"]),
    loss = loss_function,
    metrics = metrics)

In [ ]:
history = lenet_model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION[N_EPOCHS],
    verbose = 1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show()

In [ ]:
lenet_model.evaluate(val_dataset)

In [ ]:
TESTING YOUR MODEL

In [ ]:
test_image = cv2.imread("copy the path of any single image, eg a single image of happy.jpg")

im =tf.constant(test_image, dtype = tf.float32)

im = tf.expand_dims(im, axis = 0)

print(CLASS_NAMES[tf.argmax(lenet_model(im), axis = -1).numpy()[0]])

In [ ]:
# OR

plt.figure(figsize =(12,12))

for images, labels in val_dataset.take(1):
  for i in range(16):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(images[i]/225.)
    plt.title("True Label - :" + CLASS_NAMES[tf.argmax(labels[i], axis = 0).numpy()]| "\n" + "Predicted Label - :" + CLASS_NAMES[tf.argmax(lenet_model(tf.expand_dims(images[i], axis = 0)), axis = -1).numpy()[0]])
    plt.axis('off')

CONFUSION MATRIX

In [ ]:
predicted = []
labels = []

for im, label in val_dataset:
  predicted.append(lenet_model(im))
  labels.append(label.numpy())

In [ ]:
print(np.argmax(labels[:-1], axis = -1).flatten())
print(np.argmax(predicted[:-1], axis = -1).flatten())

In [ ]:
pred = np.argmax(predicted[:-1], axis = -1).flatten()
lab = np.argmax(labels[:-1], axis = -1).flatten()

In [ ]:
cm = confusion_matrix(lab, pred)
print(cm)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues')

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

plt.show()

# Define a threshold if you are doing a binary classification

DATA AUGMENTATION

In [ ]:
augment_layers = tf.keras.Sequential([
    RandomRotation(factor = (-0.025, 0.025),),
    RandomFlip(mode = 'horizontal',),
    RandomContrast(factor = 0.1)
])

def augment_layer(image, label):
  return augment_layers(image, training = True), label

In [ ]:
training_dataset = (
    train_dataset
    .map(augment_layer, num_parallel_calls = tf.data.AUTOTUNE)
    .prefetch(tf.AUTOTUNE)
)

Then retrain your model

USING CUTMIX

In [ ]:
def box(lamda):

  r_x = tf.cast(tfp.distribution.Uniform(0, CONFIGURATION["IM_SIZE"]).sample(1)[0], dtype = tf.int32)
  r_y = tf.cast(tfp.distribution.Uniform(0, CONFIGURATION["IM_SIZE"]).sample(1)[0], dtype = tf.int32)

  r_w = tf.cast(CONFIGURATION["IM_SIZE"]*tf.math.sqrt(1-lamda), dtype = tf.int32)
  r_h = tf.cast(CONFIGURATION["IM_SIZE"]*tf.math.sqrt(1-lamda), dtype = tf.int32)

  r_x = tf.clip_by_value(r_x - r_w//2, 0, CONFIGURATION["IM_SIZE"])
  r_y = tf.clip_by_value(r_y - r_h//2, 0, CONFIGURATION["IM_SIZE"])

  x_b_r = tf.clip_by_value(r_x + r_w//2, 0, CONFIGURATION["IM_SIZE"])
  y_b_r = tf.clip_by_value(r_y + r_h//2, 0, CONFIGURATION["IM_SIZE"])

  r_w = y_b_r - r_x
  if(r_w == 0):
    r_w = 1

  r_h = y_b_r - r_y
  if(r_h == 0):
    r_h = 1


  return r_y, r_x, r_h, r_w

In [ ]:
def cutmix(train_dataset_1, trian_dataset_2):
  (image_1, label_1), (image_2, label_2) = train_dataset_1, train_dataset_2

  lamda = tfp.distribution.Beta(0.2,0.2)
  lamda = lamda.sample(1)[0]

  r_y, r_x, r_h, r_w = box(lamda)

  crop_2 = tf.image.pad_to_bounding_box(image_2, r_y, r_x, r_h, r_w)

  pad_2 = tf.image.pad_to_bounding_box(crop_2, r_y, r_x, CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"])

  crop_1 = tf.image.crop_to_bounding_box(image = image_1,  r_y, r_x, r_h, r_w)

  pad_1 = tf.image.pad_to_bounding_box(crop_2, r_y, r_x, CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"])

  image = image_1 - pad_1 + pad_2

  lamda = tf.cast(1 - (r_w*r_h)/(CONFIGURATION["IM_SIZE"]*CONFIGURATION["IM_SIZE"]), dtype = tf.float34)
  label = lamda*tf.cast(label_1, dtype = tf.float32) + (1-lamda)*tf.cast(label_2, dtype = tf.float32)

  return image, label

In [ ]:
train_dataset_1 = train_dataset.map(augment_layer, num_parallel_calls = tf.data.AUTOTUNE)

train_dataset_2 = train_dataset.map(augment_layer, num_parallel_calls = tf.data.AUTOTUNE)

mixed_dataset = tf.data.Dataset.zip((train_dataset_1, train_dataset_2))

In [ ]:
training_dataset = (
    mixed_dataset
    .map(cutmix, num_parallel_calls = tf.data.AUTOTUNE)
    .prefetch(tf.AUTOTUNE))

TENSORFLOW RECORDS. Helps us build more efficient models by helping us log results of our training.

ADVANTAGES OF WORKING WITH TENSORFLOW RECORDS
1. You can store your data more efficiently
2. You can carry out preprocessing before storing the data
3. Encourages the parallelizing of read data

In [ ]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature

In [ ]:

train_dataset = (
    train_dataset
    .unbatch
)

VAL_dataset = (
    VAL_dataset
    .unbatch
)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
def create_example(image, label):
  bytes_feature = Feature(bytes_list = BytesList(value = [image]))

  int_feature = Feature(
      int64_list = Int64List(value = [label]))

  example = Example(
      features = Features(feature = {
          'images': int_feature,
          'labels': bytes_feature,
      })
  )
  return example.SerializeToString()

In [ ]:
#create a file folder on your colab files (eg tfrecords)

NUM_SHARDS = 10
PATH = 'tfrecords/shard_{:02d}.tfrecord'

In [ ]:
def encode_image(image, label):
  image = tf.image.convert_image_dtype(image, dtype = tf.uint8)
  image = tf.io.encode_jpeg(image)
  return image, tf.argmax(label)

In [ ]:
encoded_dataset = (
    train_dataset
    .map(encode_image)
)

In [ ]:
for shard_number in range(NUM_SHARD):

  sharded_dataset = (
      encoded_dataset
      .shard(NUM_SHARDS, shard_number)
      .as_numpy_iterator()
  )

  with tf.io.TFRecordWriter(PATH.format(shard_number)) as file_writer:
    for image, label in sharded_dataset:
      file_writer.write(create_example(image, label))

      file_writer.write(reord_bytes)

REUSING AND CONVERTING BACK TO TENSORFLOW DATASET

In [ ]:
recons_dataset = tf.data.TFRecordDataset(
    filenames = [PATH.format(p) for p in range(NUM_SHARDS)])

In [ ]:
def parse_tfrecords(example):

  feature_description = {
      "images": tf.io.FixedLenFeature([], tf.strings),
      "labels": tf.io.FixedLenFeature([], tf.int64)
  }

  example = tf.io.parse_single_example(example, feature_description)
  example["images"] = tf.io.decode_jpeg(example["images"], channels = 3)

  return example["images"], example["labels"]

In [ ]:
parsed_dataset = (
    recons_dataset
    .map(parse_tfrecords)
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
for i in parse_dataset.take(i):
  print(i)

In [ ]:
#Use SparseCategoricalCrossEntropy() when your labels are integers (not one-hot encoded),
#eg:
#0    class 0
#1    class 1
#2    class 2

#as against:
#[1, 0, 0]   class 0
#[0, 1, 0]   class 1
#[0, 0, 1]   class 2


loss_function = SparseCategoricalCrossEntropy()
metrics = [SparseCategoricalAccuracy(name = "accuracy")]

# then complie and train

CNN ResNet

In [ ]:
class CustomConv2D(Layer):
  def __init__(self, n_filters, kernel_size, n_strides, padding = 'valid'):
    super(CustomConv2D, self).__init__(name = 'custom_conv2d')

    self.conv = Conv2D(
        filters = n_filters
        kernel_size = kernel_size,
        activation = "relu",
        strides = n_strides,
        padding = padding
    )

    self.batch_norm BatchNormalization()

    def call(self, x, training = True):

      x = self.conv(x)
      x = self.batch_norm(x, training)

      return x


In [ ]:
class ResidualBlock(Layer):
  def __init__(self, n_channels, n_strides = 1):
    super(ResidualBlock, self).__init__(name = 'res_block')

    self.dotted = (n_strides != 1)
    self.custom_conv_1 = CustomConv2D(n_channels, 3, n_strides, padding = 'same')
     self.custom_conv_2 = CustomConv2D(n_channels, 3, 1, padding = 'same')

     self.activation = Activation('relu')

     if self.dotted:
      self.custom_conv_3 = CustomConv2D(n_channels, 1, n_strides)

  def call(self, input, training):

    x = self.custom_conv_1(input, training)
    x = self.custom_conv_2(x, training)

    if self.dotted:
      x_add = self.custom_conv_3(input, training)
      x_add = Add()([x, input])
    else:
      x_add = Add()([x, input])

    return self.activation(x_add)



In [ ]:
class ResNet34(Model):
  def__init__(self,):
    super(ResNet34, self).__init__(name = 'resnet_34')

    self.conv_1 = CustomConv2D(64, 7, 2, padding = 'same')
    self.max_pool = MaxPooling2D(3,2)

    self.conv_2_1 = ResidualBlock(64)
    self.conv_2_2 = ResidualBlock(64)
    self.conv_2_3 = ResidualBlock(64)

    self.conv_3_1 = ResidualBlock(128, 2)
    self.conv_3_2 = ResidualBlock(128)
    self.conv_3_3 = ResidualBlock(128)
    self.conv_3_4 = ResidualBlock(128)

    self.conv_4_1 = ResidualBlock(256, 2)
    self.conv_4_2 = ResidualBlock(256)
    self.conv_4_3 = ResidualBlock(256)
    self.conv_4_4 = ResidualBlock(256)
    self.conv_4_4 = ResidualBlock(256)
    self.conv_4_6 = ResidualBlock(256)

    self.conv_5_1 = ResidualBlock(521, 2)
    self.conv_5_2 = ResidualBlock(521)
    self.conv_5_3 = ResidualBlock(521)

    self.global_pool = GlobalAveragePooling2D()

    self.fc_3 = Dense(CONFIGURATION["NUM_CLASSES"], activation = 'softmax')

  def call(self, x, training = True):
    x = self.conv_1(x)
    x = self.pool1(x)

    x = self.conv_2_1(x, training)
    x = self.conv_2_2(x, training)
    x = self.conv_2_3(x, training)

    x = self.conv_3_1(x, training)
    x = self.conv_3_2(x, training)
    x = self.conv_3_3(x, training)
    x = self.conv_3_4(x, training)

    x = self.conv_4_1(x, training)
    x = self.conv_4_2(x, training)
    x = self.conv_4_3(x, training)
    x = self.conv_4_4(x, training)
    x = self.conv_4_4(x, training)
    x = self.conv_4_6(x, training)

    x = self.conv_5_1(x, training)
    x = self.conv_5_2(x, training)
    x = self.conv_5_3(x, training)

    x = self.global_pool(x)

    return self.fc_3(x)



    self.pool2 = MaxPooling2D(pool_size = pool_size)


In [ ]:
resnet_34 = ResNet34()
resnet_34(tf.zeros([1,256,256,3]), training = True)
resnet_34.summary()

In [ ]:
checkpoint_callback = ModelCheckPoint(
    'best_weight', monitor = "val_accuracy", verbose = 1, save_best_only = False, mode = 'max')

In [ ]:
metrics = [CategoricalAccuracy(name = "accuracy"),  TopKCategoricalAccuracy(k = 2, name = "top_k_accuracy")]

In [ ]:
resnet_34.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"]*10)
    loss = loss_function,
    metrics = metrics)

EVALUATION

In [ ]:
resnet_34.load_weights('best_weights')

In [ ]:
resnet_34.evaluate(val_dataset)

TRANSFER LEARNING AND FEATURE EXTRACTION

Transfer Learning can be applied in Computer Vision, Natural Langauge Processing, and Speech

In [ ]:
backbone = keras.applications.EfficientNetB4(
    include_top= False,
    weights="imagenet",
    input_shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
)

In [ ]:
# FREEZING TO PRESERVE THE WEIGHTS OF THE FEATURE LAYERS

backbone.trainable = False

In [ ]:
model = tf.Keras.Sequential([
    Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3)),
    backbone,
    GlobalAveragePooling2D(),
    Dense(CONFIGURATION['N_DENSE_1'], activation = 'relu'),
    BatchNormalization(),

    Dense(CONFIGURATION['N_DENSE_2'], activation = 'relu')
    Dense(CONFIGURATION['NUM_CLASSES'], activation = 'softmax')
])

model.summary()

In [ ]:
checkpoint_callback = ModelCheckPoint(
    'best_weight', monitor = "val_accuracy", verbose = 1, save_best_only = False, mode = 'max')

In [ ]:
loss_function = CategoricalCrossEntropy()

In [ ]:
metrics = [CategoricalAccuracy(name = "accuracy"),  TopKCategoricalAccuracy(k = 2, name = "top_k_accuracy")]

In [ ]:
model.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"])
    loss = loss_function,
    metrics = metrics)

# Then train

In [ ]:
history = lenet_model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION[N_EPOCHS],
    verbose = 1
)

EVALUATION

In [ ]:
model.evaluate(val_dataset)

In [ ]:
test_image = cv2.imread("copy the path of any single image, eg a single image of happy.jpg")

test_image = cv2.resize(test_mage,(CONFIGURATION["LEARNING_RATE"], CONFIGURATION["LEARNING_RATE"]))

im =tf.constant(test_image, dtype = tf.float32)

im = tf.expand_dims(im, axis = 0)

print(CLASS_NAMES[tf.argmax(model(im), axis = -1).numpy()[0]])

In [ ]:
plt.figure(figsize =(12,12))

for images, labels in val_dataset.take(1):
  for i in range(16):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(images[i]/225.)
    plt.title("True Label - :" + CLASS_NAMES[tf.argmax(labels[i], axis = 0).numpy()]| "\n" + "Predicted Label - :" + CLASS_NAMES[tf.argmax(lenet_model(tf.expand_dims(images[i], axis = 0)), axis = -1).numpy()[0]])
    plt.axis('off')

# Then check confusion matrix

FINETUNING TRANSFER LEARNING BUT WITH FUNCTIONAL API

In [ ]:
backbone.trainable = False

In [ ]:
input = Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3)),
x = backbone(input, training = False),
x = GlobalAveragePooling2D(x),
x = Dense(CONFIGURATION['N_DENSE_1'], activation = 'relu')(x),
x = BatchNormalization(x),

x = Dense(CONFIGURATION['N_DENSE_2'], activation = 'relu')
output = Dense(CONFIGURATION['NUM_CLASSES'], activation = 'softmax')(x)

finetuned_model = Model(input, output)

In [ ]:
finetuned_model.summary()

In [ ]:
# Then start finetuning

backbone.trainable = True

In [ ]:
input = Input(shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3)),
x = backbone(input, training = False),
x = GlobalAveragePooling2D(x),
x = Dense(CONFIGURATION['N_DENSE_1'], activation = 'relu')(x),
x = BatchNormalization(x),

x = Dense(CONFIGURATION['N_DENSE_2'], activation = 'relu')
output = Dense(CONFIGURATION['NUM_CLASSES'], activation = 'softmax')(x)

finetuned_model = Model(input, output)

In [ ]:
finetuned_model.summary()

# And complete all the other necessary steps

In [ ]:
model.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"]/100)
    loss = loss_function,
    metrics = metrics)

VISUALIZING INTERMEDIATE LAYERS

In [ ]:
vgg_backbone = keras.applications.VGG19(
    include_top=True,
    weights="imagenet",
    input_shape= (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
)

In [ ]:
vgg_backbone.summary()

In [ ]:
def is_conv(layer_name):
  if'conv' in layer_name:
    return True
  else:
    return False

In [ ]:
feature_maps = [layer.output for layer in vgg_backbone.layer[1:], if is_conv(layer.name)]
feature_map_model = Model(
    inputs = vgg_backbone.input,
    outputs = feature_maps
)

feature_map_model.summary()

In [ ]:
test_image = cv2.imread("copy the path of any single image, eg a single image of happy.jpg")

test_image = cv2.resize(test_mage,(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

im =tf.constant(test_image, dtype = tf.float32)

im = tf.expand_dims(im, axis = 0)

f_maps = feature_map_model.predict(im)

In [ ]:
for i in range(len(f_maps)):
  print(f_maps[i].shape)

FINAL VISUALIZATION

In [ ]:
for i in range(len(f_maps)):
  plt.figure(figsize = (256,256))
  f_size = f_maps[i].shape[1]
  n_channels = f_map[i].shape[3]
  joint_map = np.ones((f_size, f_size*n_channels))

  axs = plt.subplot(len(f_maps), 1, i+1)


  for j in range(n_channels):
    joint_maps[:, f_size*j:f_size*(j+1)] = f_maps[i][..., j]

  plt.imshow(joint_maps[:0:512])
  plt.axis("off")


GRAD-CAM METHOD

In [ ]:
pretrained_model.load_weights('check the directory path')

In [ ]:
test_image = cv2.imread(img_path)
test_image = cv2.resize(test_image, test_image = cv2.resize(test_mage,(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"])) )
im = tf.constant(test_image, dtype = tf.float32)
img_array = tf.expand_dims(im, axis = 0)
print(img_array.shape)


In [ ]:
preds = pretrained_model.predict(img_array)

In [ ]:
print(np.argmax(preds[0]))

In [ ]:
layer_name = "top_activation"
layer = pretrained_model.get_layer(last_conv_layer_name)
layer_model = tf.keras.Model(pretrained_model.inputs, last_conv_layer.output)

In [ ]:
last_conv_layer_name.summary()

In [ ]:
classifier_layer_names = [
    "global_average_pooling2d",
    "dense",
    "dense_1",
    "dense_2",
]

In [ ]:
classifier_input = Input(shape = (8,8,2048))
x = classifier_input

for layer in classifier_layer_names:
  x = pretrained_model.get_layer(Layer_name)(x)
  classifier_model = Model(classifier, input, x)

In [ ]:
with tf.GradientTape() as tape:
  last_conv__layer_output = last_conv_layer_mode(img_array)
  preds = classifier_model(last_conv_layer_output)
  top_pred_index = tf.argmax(preds[0])
  top_class_channel = preds[:, top_pred_index]

grads = tape.gradient(top_class_channel, last_conv_layer_output)

In [ ]:
grads.shape

ViT Models

In [ ]:
test_image = cv2.imread("copy the path of any single image, eg a single image of happy.jpg")

test_image = cv2.resize(test_mage,(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

In [ ]:
patches = tf.image.extract_patches(
    images = tf.expand_dims(test_image, axis = 0),
    sizes = [1, CONFIGURATION['PATCH_SIZE'], CONFIGURATION['PATCH_SIZE'], 1],
    strides = [1, CONFIGURATION['PATCH_SIZE'], CONFIGURATION['PATCH_SIZE'], 1],
    rates = [1, 1, 1, 1],
    padding = "valid"
)

In [ ]:
print(patches.shape)
patches = tf.reshape(patches, (patches.shape[0], -1, 768))
print(patches.shape)

In [ ]:
plt.figure(figure = (8,8))

for i in range(patches.shape[1]):
  ax = plt.subplot(16, 16, i+1)
  plt.imshow(tf.reshape(patches[0, i, :], (16, 16, 3)))
  plt.axis("off")


Patch Encoder Layer

In [ ]:
class PatchEncoder(Layer):
  def __init__(self, N_PATCHES, HIDDEN_SIZE):
    super(PatchEncoder, self).__init__(name = 'patch_encoder')

    self.liner_project = Dense(HIDDEN_SIZE)
    self.positional_embedding = Embedding(N_PATCHES, HIDDEN_SIZE)
    self.N_PATCHES = N_PATCHES

    def call(self, x):
      patches = tf.image.extract_patches(
          images = x,
          sizes = [1, CONFIGURATION['PATCH_SIZE'], CONFIGURATION['PATCH_SIZE'], 1],
          strides = [1, CONFIGURATION['PATCH_SIZE'], CONFIGURATION['PATCH_SIZE'], 1],
          rates = [1, 1, 1, 1],
          padding = "VALID"
      )
      patches = tf.reshape(patches, (tf.shape(patches), 256, patches.shape[-1]))
      embedding_input = tf.range(start = 0, limit = self.N_PATCHES, delta = 1)
      output = self.linear_projection(patches) + self.positional_embedding = Embedding(embedding_input)

      return output

In [ ]:
patch_enc = PatchEncoder(256, 768)
patch_enc(tf.zeros([1, 256, 768, 3]))

TRANSFORMER ENCODER

In [ ]:
class TransformerEncoder(Layer):
  def __init__(self, N_HEADS, HIDDEN_SIZE):
    super(TransformerEncoder, self).__init__(name = 'transformer_encoder')

    self.layer_norm_1 = LayerNormalization()
    self.layer_norm_2 = LayerNormalization()

    self.multi_head_att = MultiHeadAttention(N_HEADS, HIDDEN_SIZE)

    self.dense_1 = Dense(HIDDEN_SIZE, activation = tf.nn.gelu)
    self.dense_2 = Dense(HIDDEN_SIZE, activation = tf.nn.gelu)

  def call(self, input):
    x_1 = self.layer_norm_1(input)
    x_1 = self.multi_head_att(x_1, x_1)

    x_1 = Add([x_1, input])

    x_2 = self.layer_norm_2(x_1)
    x_2 = self.dense_1(x_2)
    output = self.dense_2(x_2)
    output = Add()([output, x_1])

    patches = tf.reshape(patches, (patches.shape[0], -1, 768))
    embedding_input = tf.range(start = 0, limit = self.N_PATCHES, delta = 1)
    output = self.linear_projection(patches) + self.positional_embedding = Embedding(embedding_input)

    return output

In [ ]:
patch_enc = TransformerEncoder(256, 768)
patch_enc(tf.zeros([1, 256, 768, 3]))

BUILDING ViT Model

In [ ]:
class ViT(Model):
  def__init__(self,N_HEADS, HIDDEN_SIZE, N_PATCHES, N_LAYERS, N_DENSE_UNITS):
    super(ViT, self).__init__(name = 'vision_transformer')
    self.N_LAYERS = N_LAYERS
    self.patch_encoder = PatchEncoder(N_PATCHES, HIDDEN_SIZE)
    self.trans_encoder = [TransformerEncoder(N_HEADS, HIDDEN_SIZE) for _ in range(N_LAYERS)]
    self.dense_1 = Dense(N_DENSE_UNITS, tf.nn.gelu)
    self.dense_2 = Dense(N_DENSE_UNITS, tf.nn.gelu)
    self.dense_3 = Dense(CONFIGURATION["NUM_CLASSES"], activation = "softmax")

  def call(self, input, training = True):

    x = self.patch_encoder(input)

    for i in range(self.N_LAYERS):
      x = self.trans_encoders[i](x)

    x = Flatten()(x)
    x = self.dense_1(x)
    x = self.dense_2(x)

    return self.dense_3(x)


In [ ]:
vit = ViT(N_HEADS = 8, HIDDEN_SIZE = 768, N_PATCHES = 256,
          N_LAYERS = 4, N_DENSE_UNITS = 1024)
vit(tf.zeros([32,256,256,3]))

In [ ]:
vit.summary()

In [ ]:
vit.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"]/100)
    loss = loss_function,
    metrics = metrics)

In [ ]:
history = vit.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION["N_EPOCHS"],
    verbose = 1
)

FINETUNING ViT USING HUGGINGFACE

In [ ]:
!pip install transformers

In [ ]:
from transformers import ViTConfig, ViTModel

# Initializing a ViT vit-base-patch16-224 style configuration
configuration = ViTConfig()

# Initializing a model (with random weights) from the vit-base-patch16-224 style configuration
model = ViTModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
# example of how to finetune

from transformers import ViTConfig, ViTModel

# Initializing a ViT vit-base-patch16-224 style configuration
configuration = ViTConfig(hidden_size = 144)

# Initializing a model (with random weights) from the vit-base-patch16-224 style configuration
model = ViTModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
print(configuration)

In [ ]:
resize_rescale_hf = tf.keras.Sequential([
    Resizing(224, 224),
    Rescaling(1./225),
    Permute((3,1,2))
])

In [ ]:
from transformers import AutoImageProcessor, TFViTModel

base_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

inputs = Input(shape = (256, 256, 3))
x = resize_rescale_hf(inputs)
x = base_model.vit(x)[0][:,0,:]

output = Dense(CONFIGURATION["NUM_CLASSES"], activation = "softmax")(x)

hf_model = tf.keras.Model(inputs = inputs, outputs = output)


In [ ]:
hf_model.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"]/100)
    loss = loss_function,
    metrics = metrics)

# do not use high learning rate when finetuning an already trained model( you can 5e-5)

In [ ]:
history = hf_model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION["N_EPOCHS"],
    verbose = 1
)

TESTING

In [ ]:
test_image = cv2.imread("copy the path of any single image, eg a single image of happy.jpg")

test_image = cv2.resize(test_mage,(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

im = tf.constant(test_image, dtype = tf.float32)
im = tf.expand_dims(im, axis = 0)

In [ ]:
hf_model(tf.expand_dims(test_image, axis = 0))

In [ ]:
hf_model.summary()

WANDB

In [ ]:
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

ONNX

In [ ]:
!pip install -U tf2onnx
!pip install onnxruntime

In [ ]:
!python -m tf2onnx.convert --saved-model name_of_saved_model/ --output name_of_new_model.onnx

WE CAN ALSO CONVERT FROM KERAS TO ONNX FORMAT

In [ ]:
hf_model.save('vit_finetuned')

In [ ]:
hf_model.save('vit_finetuned.h5')
# saving as keras model

In [ ]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec(
    (None, CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
    tf.float32, name = "input"
),)

output_path = "vit_keras.onnx"

model_proto, _ =tf2onnx.convert.from_keras(hf_model, input_signature = spec, opset = 13, out_path = output_path)
output_names = [n.name for n in model_proto.graph.output]

In [ ]:
print(output_names)

INFERENCES

In [ ]:
test_image = cv2.imread("copy the path of any single image, eg a single image of happy.jpg")

test_image = cv2.resize(test_mage,(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

im = test_image.astype(np.float32)
im = np.expand_dims(test_image, axis = 0)

In [ ]:
providers = ["CPUExecutionProvider"]
m = rt.InferenceSession(output_path, providers = providers)

t1 = time.time()
N_PREDICTIONS = 100
for _ in range(100):
  onnx_pred = m.run(output_names, {"output": im})
print("Time for a single Prediction", time.time() - t1)/N_PREDICTIONS

In [ ]:
print(onnx_pred)

CHECKING FOR LATENCY

In [ ]:
import time

In [ ]:
t1 = time.time()
hf_model(im)
print(time.time() - t1)

QUANTIZATION(A technique for performing computations and storing tensors at lower bit widths(lower precision representation) than the usual floating points which we have been working with, typically using 8-bit integars).

1. DYNAMIC QUANTIZATION
2. STATIC QUANTIZATION (BOTH ARE POST QUANTIZATION METHODS, MEANING AFTER TRAINING)
3. QAT(QUANTIZATION AWARE TRAINING), this simply means that you can improve the training accuracy of quantized models if you include the quantization error in the training phase, which enables the network to adapt to the quantized weights and activations.

QUANTIZATION ON CPU

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
model_fp32 = '/content/drive/MyDrive/Bang/vit_keras.onnx' # saved existing model
model_quant = '/content/vit_quantized.onnx'# specifying filepath where our quantized modle will be saved

quantized_model = quantized_dynamic(model_fp32, model_quant, weight_type = QuantType.QUInt8)

CHECKING THAT ACCURACY IS NOT HUGELY AFFECTING OR THAT IT IS MINIMAL

In [ ]:
def accuracy(model):
  total, acc = 0,0
  for im, label in validation_dataset.take(100):
    onnx_pred = model.run(output_names, {"input": np.array(im)})

    if (int(np.argmax(onnx_pred, axis = -1)[0][0]) = int(np.argmax(label, axis = -1[0][0]))):
      acc += 1
    total +=1
return acc/total

In [ ]:
providers = ["CUDAExecutionProvider"]
m = rt.InferenceSession("output_path before quantization", providers = providers)
m_q = rt.InferenceSession("output_path after quantization", providers = providers)

print(accuracy(m))
print(accuracy(m_q))

QAT(QUANTIZATION AWARE TRAINING)

In [ ]:
!pip install -q tensorflow-model-optimization

In [ ]:
def apply_quantization_to_conv(layer):
  if "conv" in layer.name:
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

In [ ]:
quant_aware_eff = tf.keras.models.clone_model(
    pretrained_model, clone_function = apply_quantization_to_conv
)

In [ ]:
quant_aware_eff.summary()

In [ ]:
quant_aware_model = tfmot.quantization.keras.quantize_apply(quant_aware_eff)
quant_aware_model.summary()

In [ ]:
quant_aware_model.compile(
    optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"])
    loss = loss_function,
    metrics = metrics)

In [ ]:
history = quant_aware_model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION["N_EPOCHS"],
    verbose = 1
)

QUANTIZING AN ALREADY TRAINED MODEL USING TENSORFLOW LITE. (suitable for mobile and other edge devices)

In [ ]:
# Load your trained model and evaluate on the Val dataset to  get the accuracy

In [ ]:
def representative_data_gen():
  for input_value, j in training_dataset.take(20):
    yield [input_value]

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(name_of_your_model)
converter.optimizations = [tf.lite.Optimize.Default]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representativae_data_gen

# do not specify representativae_data_gen if you are performing a dynamic quantization

In [ ]:
tflite_model = converter.convert()

SAVE IN TFLITE FORMAT

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("file_path")
tflite_models_dir.mkdir(exist_ok = True, parents = True)

tflite_model_file = tflite_models_dir/'eff_model.tflite'
tflite_model_file.write_bytes(tflite_model)



INSTALL TFLITE RUNTIME

In [ ]:
!pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [ ]:
import tflite_runtime as tflite
import numpy as np
import cv2

In [ ]:
test_image = cv2.imread("copy the file path of any single image, eg a single image of happy.jpg")

test_image = cv2.resize(test_mage,(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))
test_image = np.expand_dims(test_image, axis = 0)

#print(CONFIGURATION['CLASS_NAMES'][tf.argmax(pretrained_model(im), axis = -1).numpy()[0]])

In [ ]:
interpreter = tf.lite.Interpreter(model_path = "copy the file path of eff_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

#test_image = im.numpy().astype(input_details["dtype"])
interpreter.set_tensor(input_details["index"], test_image)

interpreter.invoke()

output = interpreter.get_tensor(output_details["index"])[0]

In [ ]:
print(CONFIGURATION['CLASS_NAMES'][np.argmax(output)])

In [ ]:
def accuracy(model_path):
  total, correct = 0,0
  interpreter = tf.lite.Interprater(model_path = "copy the file path of eff_model.tflite")
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  for im, label in validation_dataset.take:
    test_image = im.numpy().astype(input_details["dtype"])

    interpreter.set_tensor(input_details["index"], test_image)

    interpreter.invoke()

    output = interpreter.get_tensor(output_details["index"])[0]

    if(int(np.argmax(output)) == int(np.argmax(label, axis = -1)[0])):
      correct += 1
    total += 1
  return correct/total

In [ ]:
accuracy("copy the file path of eff_model.tflite")

APIs

signup on web.postman.co and click on workplace. locat the "GET" (which is a drop down) and the URL beside it.

The "GET" is used to retrieve data
The "POST" is used to submit an entity to the specific resourse
The "PUT" is used to update

BUILDING APIs USING FASTAPI

use the anaconda

In [ ]:
pip install "fastapi[all]"

In [ ]:
import fastapi

In [ ]:
fastapi.__version__

DEALING WITH FASTAPIs OF DIFFERENT VERSIONS AND DIFFERENT MODELS

In [ ]:
apt install python3-venv
#then cd into your model directory and create a virtual environment

In [ ]:
#virtual environment

python3 -m venv venv-emotions-detection

In [ ]:
cd venv-emotions-detection/